In [71]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,4 hours 49 mins
H2O cluster timezone:,Asia/Colombo
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.11
H2O cluster version age:,2 months and 2 days
H2O cluster name:,H2O_from_python_sajeewa_ykd1d6
H2O cluster total nodes:,1
H2O cluster free memory:,1.867 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [72]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator as glm
from builtins import range

In [74]:
train = h2o.import_file('data/train_preprocess_feature_engineering.csv')
# train['donated'] = train['donated'].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [75]:
predictors = ["mon_since_last", "num_donations", "mon_since_first", "log_difference", "log_donation_period", "log_ratio", "old_good_donor", "old_bad_donor", "good_donor", "bad_donor", "quitted"]
response_col = "donated"

train["old_good_donor"] = train["old_good_donor"].asfactor()
train["old_bad_donor"] = train["old_bad_donor"].asfactor()
train["good_donor"] = train["good_donor"].asfactor()
train["bad_donor"] = train["bad_donor"].asfactor()
train["quitted"] = train["quitted"].asfactor()

In [79]:
glm_model = glm(family= "binomial", lambda_search=True, solver='COORDINATE_DESCENT')
glm_model.train(predictors, response_col, training_frame= train)
glm_regularization_path = glm.getGLMRegularizationPath(glm_model)

glm_model2 = glm.makeGLMModel(model=glm_model, coefs=glm_regularization_path['coefficients'][10])
dev1 = glm_regularization_path['explained_deviance_train'][10]
p = glm_model2.model_performance(train)
dev2 = 1-p.residual_deviance()/p.null_deviance()
print(dev1," =?= ",dev2)

assert abs(dev1 - dev2) < 1e-6

for l in range(0,len(glm_regularization_path['lambdas'])):
    glm_model = glm(family='binomial',lambda_search=False,Lambda=glm_regularization_path['lambdas'][l],solver='COORDINATE_DESCENT')
    glm_model.train(predictors, response_col, training_frame= train)
    cs = glm_regularization_path['coefficients'][l]
    cs_norm = glm_regularization_path['coefficients_std'][l]
    diff = 0
    diff2 = 0
    for n in cs.keys():
        diff = max(diff,abs((cs[n] - glm_model.coef()[n])))
        diff2 = max(diff2,abs((cs_norm[n] - glm_model.coef_norm()[n])))
    print(diff)
    print(diff2)
#     assert diff < 1e-2
#     assert diff2 < 1e-2
    p = glm_model.model_performance(train)
    devm = 1-p.residual_deviance()/p.null_deviance()
    devn = glm_regularization_path['explained_deviance_train'][l]
    print(devm)
    print(devn)
    assert abs(devm - devn) < 1e-4

glm Model Build progress: |███████████████████████████████████████████████| 100%
(0.08922469895274854, ' =?= ', 0.08922469895271168)
glm Model Build progress: |███████████████████████████████████████████████| 100%
0
0
0.0
0.0
glm Model Build progress: |███████████████████████████████████████████████| 100%
6.22241147497e-12
5.99367777632e-13
0.00890403964778
0.00890403964755
glm Model Build progress: |███████████████████████████████████████████████| 100%
8.03770975383e-05
7.71625728137e-06
0.0184356969026
0.0184367068568
glm Model Build progress: |███████████████████████████████████████████████| 100%
0.000313116827068
3.6205377152e-05
0.0302130156973
0.0302105406919
glm Model Build progress: |███████████████████████████████████████████████| 100%
0.000435375743615
6.39682113528e-05
0.0419573420058
0.0419589350317
glm Model Build progress: |███████████████████████████████████████████████| 100%
3.94206983452e-05
8.73334980196e-05
0.0524831034471
0.0524971614737
glm Model Build progress: |█

0.151101891188
0.151097448815
glm Model Build progress: |███████████████████████████████████████████████| 100%
0.116076331495
0.115524732309
0.151163905291
0.151159537287
glm Model Build progress: |███████████████████████████████████████████████| 100%
0.126100543283
0.125916166136
0.151216145388
0.151211731649
glm Model Build progress: |███████████████████████████████████████████████| 100%
0.138000251679
0.13787797203
0.151259899543
0.151255409752
glm Model Build progress: |███████████████████████████████████████████████| 100%
0.151108492532
0.15079520477
0.15129648454
0.151292581133


In [80]:
test = h2o.import_file('data/test_preprocess_feature_engineering.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [83]:
test['donated'] = None

test["old_good_donor"] = test["old_good_donor"].asfactor()
test["old_bad_donor"] = test["old_bad_donor"].asfactor()
test["good_donor"] = test["good_donor"].asfactor()
test["bad_donor"] = test["bad_donor"].asfactor()
test["quitted"] = test["quitted"].asfactor()

In [84]:
test

mon_since_last,num_donations,mon_since_first,log_difference,log_donation_period,log_ratio,old_good_donor,old_bad_donor,good_donor,bad_donor,quitted,donated
2,12,52,1.59571,0.188668,0.0370456,1,0,1,0,0,nan
21,7,38,1.3585,0.156214,0.364609,0,1,0,1,0,nan
4,1,4,0,0.201424,0.526589,0,0,0,0,0,nan
11,11,38,1.46608,0.226525,0.226525,0,1,0,1,0,nan
4,12,34,1.4893,0.264117,0.105464,0,0,0,0,0,nan
3,21,42,1.54519,0.340368,0.066717,1,0,0,0,0,nan
4,2,4,0,0.340368,0.526589,0,0,0,0,0,nan
14,1,14,0,0.066717,0.526589,0,0,0,1,1,nan
23,2,87,1.64372,0.0224738,0.210725,0,1,0,1,1,nan
14,4,64,1.59571,0.058858,0.180508,0,1,0,1,0,nan


In [85]:
predictions = glm_model.predict(test)

glm prediction progress: |████████████████████████████████████████████████| 100%


In [87]:
y = h2o.import_file('data/test_data.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [56]:
ids=y['C1']

In [88]:
train

mon_since_last,num_donations,mon_since_first,donated,log_difference,log_donation_period,log_ratio,old_good_donor,old_bad_donor,good_donor,bad_donor,quitted
2,50,98,1,1.71824,0.34518,0.0200013,1,0,1,0,0
0,13,28,1,1.47414,0.323074,0,1,0,1,0,0
1,16,35,1,1.5163,0.319528,0.0277814,1,0,1,0,0
2,20,45,1,1.56532,0.313149,0.0425662,1,0,1,0,0
1,24,77,0,1.67594,0.240052,0.0128209,1,0,1,0,0
4,4,4,0,0,0.526589,0.526589,0,0,0,0,0
2,7,14,1,1.27115,0.340368,0.125338,0,0,1,0,0
1,12,35,0,1.5163,0.258356,0.0277814,1,0,1,0,0
5,46,98,1,1.71259,0.325589,0.048563,0,1,0,1,0
0,3,4,0,0.959135,0.44444,0,0,0,1,0,0


In [66]:
predictions_class1 = [ids, 'Made Donation in March 2007':predictions['p1']]

In [89]:
h2o.export_file(predictions['p1'], "submission_h2o_glm_features_engineered.csv", force=True)

Export File progress: |███████████████████████████████████████████████████| 100%


In [70]:
h2o.export_file(ids, "ids.csv", force=True)

Export File progress: |███████████████████████████████████████████████████| 100%
